In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.ensemble import GradientBoostingRegressor  # Change to your best model

# Train final model on full training data
train = pd.read_csv('data/processed_train.csv')
X = train.drop(['Id', 'SalePrice'], axis=1)
y = train['SalePrice']

# Use your best model from previous notebook
final_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
final_model.fit(X, y)

# Save the model
os.makedirs('models', exist_ok=True)
with open('models/house_price_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

# Create a prediction function for new houses
def predict_price(house_features):
    """
    Predict the price of a house given its features.
    
    Parameters:
    house_features (dict): Dictionary of house features
    
    Returns:
    float: Predicted house price
    """
    # Convert input to DataFrame
    house_df = pd.DataFrame([house_features])
    
    # Ensure all features match the training data
    missing_cols = set(X.columns) - set(house_df.columns)
    for col in missing_cols:
        house_df[col] = 0
    
    # Reorder columns to match training data
    house_df = house_df[X.columns]
    
    # Make prediction (log scale)
    log_price = final_model.predict(house_df)[0]
    
    # Convert back from log scale
    price = np.expm1(log_price)
    
    return price

# Example usage
if __name__ == "__main__":
    # Example house - modify with actual features from your dataset
    example_house = {
        'GrLivArea': np.log1p(1500),  # Log-transformed living area
        'TotalBsmtSF': np.log1p(1000),  # Log-transformed basement area
        'OverallQual': 7,
        # Add other important features based on your model
        'TotalSF': np.log1p(2500),  # Total square footage
        # Fill in remaining features with appropriate values or zeros
    }
    
    predicted_price = predict_price(example_house)
    print(f"Predicted House Price: ${predicted_price:,.2f}")

Predicted House Price: $96,964.11


/Users/chrisjagielski/Documents/git/ai_predictive_model_for_house_prices/venv/lib/python3.7/site-packages/ipykernel_launcher.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
